### 주가정보 획득
* cj제일제당을 해보자 (097950)  
* 네이버 금융을 검색하면 주가정보를 볼 수 있다 > '시세' 부분에 들어감
 - https://finance.naver.com/item/sise_day.nhn?code=097950&page=1

 - *주의* -  위 url은 네이버 금융 > cj제일제당 > '시세' > 페이지소스보기 > '일별'이라는 단어로 Ctrl+F 검색하여 나오는 url 임. 'iframe'은 소스코드에 또 새로운 걸 이중으로 입힌 것으로, 복잡하게 되어있는데 위 url로 들어가야 편리하게 분석할 수 있음
 


In [15]:
import sqlite3  # 크롤링한 데이터를 저장함. 주식은 시간이 지나면 바뀌는 데이터이므로 저장해놓고 업데이트 해야함.
import requests # 내부 source 를 가져오는 라이브러리
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [16]:
# 함수 선언부 url 을 매개변수화 - url이라는 변수를 가져올건데 일단 공백으로.
def stockGet(url=''):    
    # url 가져와서 파싱
    src=requests.get(url)
    result=src.content.strip()    
    html=BeautifulSoup(result,'html.parser')
    tgts=html.select('table.type2 tr')
    #len(tgt)
    res=[]
    #tgts 가 16개 씩 나옴 1 page
    for tgt in tgts:
        if(tgt.select('td .p10')):# 날짜 없는 줄은 제거       # 띄어쓰기 = 그 밑 다 가져오라는
            sday=tgt.select('td .p10')[0].text.replace(".","")

            endp=tgt.select('td.num')[0].text.replace(",","")
            startp=tgt.select('td.num')[2].text.replace(",","")
            maxp=tgt.select('td.num')[3].text.replace(",","")
            minp=tgt.select('td.num')[4].text.replace(",","")
            vol=tgt.select('td.num')[5].text.replace(",","")
            res.append((sday,startp,maxp,minp,endp,vol))
    # 10개 반환 (한 페이지당 10개씩)
    return res


In [23]:
def addData(data):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #  \는 줄이 길어져서 나눈 것. 그리고 cj_jaeil은 테이블명
        sql = 'insert into cj_jaeil \
        (SIDX,S_CODE,SCOMP,SDAY,OPENP,HIGHP,LOWP,CLOSEP,VOL) \
        values (?,?,?,?,?,?,?,?,?)'
                       # ?,?는 알아서 채우라는 것
        ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
        cur.execute(sql,data)   #이 sql문대로 실행함
        conn.commit()
        cur.close()


In [24]:
dbname='cj_jaeil.db'     # sqlite에 만든 db이름!
def selData(whr=' where SIDX=0'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select SIDX from cj_jaeil '+whr     # from 뒤는 테이블 명
        cur.execute(sql)
        rows=cur.fetchall()
        cur.close()
    return rows

 - 이 코드가 같은 것임. ( with sqlite3.connect(dbname) as conn : ~ )
dbname = 'stockdata.db'
conn= sqlite3.connect(dbname)
cur = conn.cursor()
sql='select * from StockDay'   #sql문
result=cur.execute(sql)    # sql문대로 가져오기
rows=result.fetchall()   #fetch 가져오는거
rows

#result 말고 cur.execute(sql) 하고 rows=cur.fetchall()해도 ok

In [25]:
urlsrc='https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}'
sCode='097950'   # 해당 페이지에 코드가 있음
sComp='CJ제일제당'    # 회사 이름
for i in range(1,10):       #10페이지까지 긁어옴
    print("#"*30)
    url=urlsrc.format(sCode,i)
    print(url)
    # 데이터 10개 가져오기
    rs=stockGet(url)
    
    for r in rs:
        #데이터별 고유코드만들기  --> 중복방지용
        sidx=r[0]+sCode
        ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
        data=[sidx,sCode,sComp,r[0],r[1],r[2],r[3],r[4],r[5]]
        #데이터 입력하기! 중복데이터 확인
        preData=selData(' where SIDX='+sidx)
        if(preData):                               #만약 중복되면 안 넣고 중복 안 되면 넣음.
            print("중복데이터 입니다.",preData)
        else:
            #print(r)
            addData(data)

##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=1
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=2
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=3
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=4
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=5
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=6
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=7
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=8
##############################
https://finance.naver.com/item/sise_day.nhn?code=097950&page=9


In [6]:
selData()

[]

In [55]:
# 아.. 신세계 완료 ....